# 꽃 이미지 분류 딥러닝 프로젝트

이 노트북은 꽃 이미지를 분류하는 CNN 모델을 구축하고 학습하는 과정을 담고 있습니다.

## 프로젝트 개요
- **목표**: 5가지 꽃 종류(daisy, dandelion, rose, sunflower, tulip)를 분류하는 모델 개발
- **방법**: Convolutional Neural Network (CNN) 사용
- **데이터**: 꽃 이미지 데이터셋
- **프레임워크**: TensorFlow/Keras

## 프로젝트 구조
1. 라이브러리 임포트 및 환경 설정
2. 데이터 전처리 함수 정의
3. 데이터셋 구성 및 분할
4. 딥러닝 모델 구축
5. 모델 학습 및 저장


## 1. 라이브러리 임포트 및 환경 설정

필요한 라이브러리들을 임포트합니다.


In [ ]:
# 딥러닝 관련 라이브러리
from keras.preprocessing import image
from keras.models import load_model 
from keras import models, layers
import tensorflow as tf
import keras

# 데이터 처리 및 시각화 라이브러리
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
import PIL.Image as pilimg 

# 파일 시스템 관련 라이브러리
import os
import shutil
import random 
import imghdr
import pickle

print("라이브러리 임포트 완료!")
print(f"TensorFlow 버전: {tf.__version__}")
print(f"Keras 버전: {keras.__version__}")


## 2. 데이터 전처리 함수 정의

이미지 데이터셋을 정리하고 분할하기 위한 함수들을 정의합니다.

### 주요 기능:
- **이미지 리네이밍**: 클래스별로 일관된 파일명으로 변경
- **데이터 분할**: Train/Validation/Test 세트로 분할 (50:25:25 비율)
- **디렉토리 구성**: 모델 학습에 적합한 폴더 구조 생성


In [ ]:
def rename_images_in_class_folder(src_class_dir, class_name, dest_dir):
    """
    특정 클래스 폴더의 이미지들을 'class_name.인덱스.확장자' 형식으로 리네이밍
    
    Args:
        src_class_dir: 원본 클래스 폴더 경로
        class_name: 클래스명 (예: 'daisy')
        dest_dir: 리네이밍된 이미지가 저장될 폴더
    """
    os.makedirs(dest_dir, exist_ok=True)

    # 이미지 파일만 필터링
    files = [f for f in os.listdir(src_class_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    files.sort()

    for idx, fname in enumerate(files):
        src_path = os.path.join(src_class_dir, fname)
        ext = os.path.splitext(fname)[1].lower()
        new_name = f"{class_name}.{idx}{ext}"
        dst_path = os.path.join(dest_dir, new_name)
        shutil.copyfile(src_path, dst_path)

    print(f"✅ {class_name} 리네임 완료: {len(files)}개 처리됨.")


def rename_all_classes(original_root_dir, renamed_root_dir):
    """
    모든 클래스의 이미지들을 리네이밍하여 하나의 폴더에 통합
    
    Args:
        original_root_dir: 원본 데이터셋 폴더 (클래스별 하위 폴더 포함)
        renamed_root_dir: 리네이밍된 이미지들이 저장될 폴더
    """
    classes = ["daisy", "dandelion", "rose", "sunflower", "tulip"]

    # 기존 폴더가 있으면 삭제 후 새로 생성
    if os.path.exists(renamed_root_dir):
        shutil.rmtree(renamed_root_dir)
    os.makedirs(renamed_root_dir, exist_ok=True)

    for class_name in classes:
        src_class_dir = os.path.join(original_root_dir, class_name)
        rename_images_in_class_folder(src_class_dir, class_name, renamed_root_dir)

print("이미지 리네이밍 함수 정의 완료!")


In [ ]:
def copy_images_by_class(class_name, original_dataset_dir, dest_dirs, split_ratio=(0.5, 0.25, 0.25)):
    """
    특정 클래스의 이미지들을 Train/Validation/Test로 분할 복사
    
    Args:
        class_name: 클래스명
        original_dataset_dir: 리네이밍된 이미지들이 있는 폴더
        dest_dirs: [train_dir, val_dir, test_dir] 리스트
        split_ratio: 분할 비율 (기본값: 50:25:25)
    """
    # 해당 클래스의 이미지 파일들 찾기
    image_files = [
        f for f in os.listdir(original_dataset_dir)
        if f.startswith(f"{class_name}.") and f.lower().endswith(('.jpg', '.jpeg', '.png')) 
        and os.path.isfile(os.path.join(original_dataset_dir, f))
    ]
    image_files.sort()
    random.shuffle(image_files)  # 랜덤 셔플

    # 분할 인덱스 계산
    total = len(image_files)
    train_end = int(total * split_ratio[0])
    val_end = train_end + int(total * split_ratio[1])

    # 분할된 파일 리스트
    splits = [image_files[:train_end], image_files[train_end:val_end], image_files[val_end:]]

    # 각 세트로 파일 복사
    for split, dst_dir in zip(splits, dest_dirs):
        os.makedirs(dst_dir, exist_ok=True)
        for fname in split:
            src = os.path.join(original_dataset_dir, fname)
            dst = os.path.join(dst_dir, fname)
            shutil.copyfile(src, dst)


def ImageCopy(renamed_dataset_dir, base_dir):
    """
    리네이밍된 이미지들을 Train/Validation/Test 폴더로 분할하여 복사
    
    Args:
        renamed_dataset_dir: 리네이밍된 이미지들이 있는 폴더
        base_dir: Train/Val/Test 폴더들이 생성될 기본 경로
    """
    categories = ["daisy", "dandelion", "rose", "sunflower", "tulip"]
    sets = ["train", "validation", "test"]

    # 기존 폴더 삭제 후 새로 생성
    if os.path.exists(base_dir):
        shutil.rmtree(base_dir)
    for set_name in sets:
        for category in categories:
            os.makedirs(os.path.join(base_dir, set_name, category), exist_ok=True)

    # 폴더 경로 설정
    train_dir = os.path.join(base_dir, "train")
    val_dir = os.path.join(base_dir, "validation")
    test_dir = os.path.join(base_dir, "test")

    # 각 클래스별로 이미지 분할 복사
    for category in categories:
        print(f"🔄 {category} 분할 중...")
        copy_images_by_class(
            class_name=category,
            original_dataset_dir=renamed_dataset_dir,
            dest_dirs=[
                os.path.join(train_dir, category),
                os.path.join(val_dir, category),
                os.path.join(test_dir, category)
            ],
            split_ratio=(0.5, 0.25, 0.25)
        )

    print("\n✅ 이미지 분할 복사 완료!\n")

    # 결과 요약 출력
    for set_name in sets:
        for category in categories:
            dir_path = os.path.join(base_dir, set_name, category)
            count = len(os.listdir(dir_path))
            print(f"📁 {set_name}/{category}: {count}개")

print("데이터 분할 함수 정의 완료!")


## 3. 데이터셋 구성 및 분할 실행

실제로 데이터 전처리를 수행합니다.

### 처리 과정:
1. **1단계**: 클래스별 이미지들을 `class_name.index.ext` 형식으로 리네이밍
2. **2단계**: 리네이밍된 이미지를 Train(50%) / Validation(25%) / Test(25%) 비율로 분할

### 폴더 구조:
```
flowers_small/
├── train/
│   ├── daisy/
│   ├── dandelion/
│   ├── rose/
│   ├── sunflower/
│   └── tulip/
├── validation/
│   └── (동일한 구조)
└── test/
    └── (동일한 구조)
```


In [ ]:
# 데이터셋 경로 설정
original_dataset_dir = "../../data/flowers"           # 원본 데이터셋 폴더 (클래스별 하위 폴더 있음)
renamed_root = "../../data/flowers_renamed"           # 리네이밍된 이미지들이 저장될 위치
base_dir = "../../data/flowers_small"                 # 최종 분할된 train/val/test 폴더 생성 위치

print("📁 데이터셋 경로 설정 완료!")
print(f"원본 데이터: {original_dataset_dir}")
print(f"리네이밍 저장: {renamed_root}")
print(f"분할 저장: {base_dir}")


In [ ]:
# 1단계: 클래스별 이미지들을 daisy.0.jpg 형식으로 리네이밍 + 통합
print("🔄 1단계: 이미지 리네이밍 시작...")
rename_all_classes(original_dataset_dir, renamed_root)
print("✅ 1단계 완료: 모든 이미지가 리네이밍되었습니다!")


In [ ]:
# 2단계: 리네이밍된 이미지를 50:25:25 비율로 train/validation/test 분할 복사
print("🔄 2단계: 데이터 분할 시작...")
ImageCopy(renamed_root, base_dir)
print("✅ 2단계 완료: 데이터 분할이 완료되었습니다!")


## 4. 딥러닝 모델 구축 및 학습

CNN(Convolutional Neural Network) 모델을 구축하고 학습합니다.

### 모델 구조:
1. **데이터 증강(Data Augmentation)**: RandomFlip, RandomRotation, RandomZoom
2. **전처리**: Rescaling (0-1 정규화)
3. **CNN 레이어들**: 
   - Conv2D (32, 64, 32 필터) + MaxPooling2D
   - Flatten + Dropout (0.5)
   - Dense (128, 64 유닛) + 출력층 (5 클래스)

### 학습 설정:
- **옵티마이저**: Adam
- **손실 함수**: Sparse Categorical Crossentropy
- **평가 지표**: Accuracy
- **에포크**: 30


In [ ]:
# 분할된 데이터셋 경로 설정
train_dir = os.path.join(base_dir, "train")
validation_dir = os.path.join(base_dir, "validation")
test_dir = os.path.join(base_dir, "test")

print("📁 훈련 데이터 경로 설정 완료!")
print(f"Train: {train_dir}")
print(f"Validation: {validation_dir}")
print(f"Test: {test_dir}")


In [ ]:
# 데이터 증강 레이어 정의
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal", input_shape=(180, 180, 3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

print("✅ 데이터 증강 레이어 정의 완료!")
print("- RandomFlip: 수평 뒤집기")
print("- RandomRotation: ±10% 회전")
print("- RandomZoom: ±10% 확대/축소")


In [ ]:
# CNN 모델 구축
model = models.Sequential()

# 전처리 및 데이터 증강
model.add(layers.Rescaling(1./255))  # 픽셀 값을 0-1로 정규화
model.add(data_augmentation)

# 합성곱 레이어들
model.add(layers.Conv2D(32, (3, 3), activation="relu"))
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Conv2D(64, (3, 3), activation="relu"))
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Conv2D(32, (3, 3), activation="relu"))
model.add(layers.MaxPooling2D(2, 2))

# 완전연결 레이어들
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))  # 과적합 방지
model.add(layers.Dense(128, activation="relu"))
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(5, activation="softmax"))  # 5개 클래스 분류

# 모델 컴파일
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",  # 정수 라벨용 다중 분류
    metrics=["accuracy"]
)

print("✅ CNN 모델 구축 완료!")
model.summary()


In [ ]:
# 훈련 데이터셋 생성
train_ds = keras.utils.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,  # 훈련 데이터의 20%를 추가 검증용으로 사용
    seed=123,
    subset="training",
    image_size=(180, 180),
    batch_size=16
)

# 검증 데이터셋 생성
validation_ds = keras.utils.image_dataset_from_directory(
    validation_dir,
    validation_split=0.2,
    seed=123,
    subset="validation",
    image_size=(180, 180),
    batch_size=16
)

print("✅ 데이터셋 로드 완료!")
print(f"훈련 데이터셋: {train_ds}")
print(f"검증 데이터셋: {validation_ds}")

# 클래스 이름 확인
class_names = train_ds.class_names
print(f"클래스: {class_names}")


## 5. 모델 학습 및 저장

실제로 모델을 학습시키고 저장합니다.

### 학습 과정:
- **에포크**: 30회 반복 학습
- **데이터**: 증강된 훈련 데이터 + 검증 데이터
- **결과**: 각 에포크마다 훈련/검증 정확도와 손실 출력
- **저장**: 학습 완료된 모델을 `flowers_model.keras` 파일로 저장


In [ ]:
# 모델 학습 시작
print("🚀 모델 학습 시작...")
print("※ 30 에포크 학습에는 시간이 걸릴 수 있습니다.")

history = model.fit(
    train_ds, 
    validation_data=validation_ds,
    epochs=30,
    verbose=1  # 학습 과정 출력
)

print("✅ 모델 학습 완료!")

# 모델 저장
model_save_path = "flowers_model.keras"
model.save(model_save_path)
print(f"💾 모델이 저장되었습니다: {model_save_path}")

# 최종 성능 출력
final_train_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]
print(f"\n📊 최종 성능:")
print(f"- 훈련 정확도: {final_train_acc:.4f}")
print(f"- 검증 정확도: {final_val_acc:.4f}")


## 6. 결론 및 다음 단계

### 완료된 작업:
✅ **데이터 전처리**: 이미지 리네이밍 및 Train/Val/Test 분할  
✅ **모델 구축**: CNN 아키텍처 설계 및 데이터 증강 적용  
✅ **모델 학습**: 30 에포크 학습 및 성능 평가  
✅ **모델 저장**: `flowers_model.keras` 파일로 저장  

### 다음 단계 제안:
1. **성능 개선**:
   - 하이퍼파라미터 튜닝 (학습률, 배치 크기, 에포크 수)
   - 더 복잡한 모델 아키텍처 시도 (ResNet, EfficientNet 등)
   - 더 다양한 데이터 증강 기법 적용

2. **모델 평가**:
   - Test 데이터셋으로 최종 성능 평가
   - Confusion Matrix 생성
   - 잘못 분류된 이미지 분석

3. **모델 활용**:
   - 새로운 꽃 이미지 예측 함수 구현
   - 웹 애플리케이션 또는 API로 배포
   - 실시간 이미지 분류 시스템 구축

### 학습한 주요 개념:
- **CNN 아키텍처**: Conv2D, MaxPooling2D, Dense 레이어
- **데이터 증강**: 과적합 방지 및 일반화 성능 향상
- **데이터 전처리**: 체계적인 데이터 관리 및 분할
- **모델 저장/로드**: 학습된 모델의 재사용
